<a href="https://colab.research.google.com/github/hieplh/PPR501/blob/main/Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary modules and libraries

In [ ]:
# importing all necessary libraries
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from tqdm.notebook import tqdm as tqdm

import plotly.graph_objs as go #visualization library
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf #autocorrelation test
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller #stationarity test
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from datetime import datetime, timedelta
import statsmodels.api as sm
import gc
from pylab import rcParams
import random
import os
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import time
import numpy as np
import pandas as pd
import os
import time
import matplotlib.dates as mdates
import gc
import lightgbm as lgb
import time

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

Install and import some libraries necessary

# Load the data

In [ ]:
sell_prices = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/sell_prices.csv')
df_val = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/sales_train_validation.csv')
calender = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/calendar.csv')
train_sales = pd.read_csv('https://media.githubusercontent.com/media/minhhieu9800/Python-MSE/main/Data/sales_train_evaluation.csv')

Tải tập dữ liệu từ đường dẫn github và gán vào biến

# Data Exploration

In [ ]:
train_sales.isnull().sum().sort_values(ascending = False)
sell_prices.isnull().sum().sort_values(ascending = False)
calender.isnull().sum().sort_values(ascending = False)

event_name_2    1964
event_type_2    1964
event_name_1    1807
event_type_1    1807
date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

Chuyển đổi tất cả dữ liệu nào không có giá trị thành true, dữ liệu có giá trị thành false

Sau đấy chuyển đổi tiếp true, false thành con số, với true = 1, false = 0

Sắp xếp theo thứ tự giảm dần

In [ ]:
holiday = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'SuperBowl', 'PresidentsDay', \
           'StPatricksDay', 'Easter', 'Cinco De Mayo', 'IndependenceDay', \
           'EidAlAdha', 'Thanksgiving', 'Christmas']
weekend = ['Saturday', 'Sunday']

def is_holiday(x):
    if x in holiday:
        return 1
    else:
        return 0

def is_weekend(x):
    if x in weekend:
        return 1
    else:
        return 0

Tạo 2 mảng là holiday và weekend

Tạo 2 phương thức để kiểm tra tham số đầu vào nếu 2 mảng đã tạo trên có giá trị đấy thì sẽ trả về là 1, ngược lại là 0

In [ ]:
calender['is_holiday_1'] = calender['event_name_1'].apply(is_holiday)
calender['is_holiday_2'] = calender['event_name_2'].apply(is_holiday)
calender['is_holiday'] = calender[['is_holiday_1','is_holiday_2']].max(axis=1)
calender['is_weekend'] = calender['weekday'].apply(is_weekend)

```
calender['is_holiday_1'] = calender['event_name_1'].apply(is_holiday)
```
*   Lấy giá trị của cột event_name_1 của tập dữ liệu calender và truyền vào tham số của phương thức is_holiday 
*   Gán giá trị trả về của phương thức is_holiday vào cột is_holiday_1 của tập dữ liệu calender

```
calender['is_holiday_2'] = calender['event_name_2'].apply(is_holiday)
```
*   Lấy giá trị của cột event_name_2 của tập dữ liệu calender và truyền vào tham số của phương thức is_holiday 
*   Gán giá trị trả về của phương thức is_holiday vào cột is_holiday_2 của tập dữ liệu calender

```
calender['is_holiday'] = calender[['is_holiday_1','is_holiday_2']].max(axis=1)
```
*   Lấy giá trị lớn nhất giữa 2 cột is_holiday_1 và is_holiday_2 của tập dữ liệu calender
*   Gán giá trị đã lấy được vào cột is_holiday của tập dữ liệu calender

```
calender['is_weekend'] = calender['weekday'].apply(is_weekend)
```
*   Lấy giá trị của cột weekday của tập dữ liệu calender và truyền vào tham số của phương thức is_weekend
*   Gán giá trị trả về của phương thức is_weekend vào cột is_weekend của tập dữ liệu calender

In [ ]:
calender.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0


In ra 5 dòng đầu tiên của tập dữ liệu calender

In [ ]:
calender = calender.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [ ]:
sell_prices.describe()

,wm_yr_wk,sell_price
count,6.841121e+06,6.841121e+06
mean,1.138294e+04,4.410952e+00
std,1.486100e+02,3.408814e+00
min,1.110100e+04,1.000000e-02
25%,1.124700e+04,2.180000e+00
50%,1.141100e+04,3.470000e+00
75%,1.151700e+04,5.840000e+00
max,1.162100e+04,1.073200e+02
